<a href="https://colab.research.google.com/github/RuwanPradeep0/Object-Detection-in-Trafic-Computer-vision-project-EC9580/blob/main/Detect_and_classify_Traffic_Objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets -q

In [2]:
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/saumyapatel/traffic-vehicles-object-detection/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ruwanpradeep
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/saumyapatel/traffic-vehicles-object-detection


100%|██████████| 607M/607M [00:34<00:00, 18.5MB/s]


In [6]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Define directories (update as necessary for your setup)
base_dir = '/content/traffic-vehicles-object-detection/Traffic Dataset'
train_images_dir = os.path.join(base_dir, 'images', 'train')
val_images_dir = os.path.join(base_dir, 'images', 'val')
train_labels_dir = os.path.join(base_dir, 'labels', 'train')
val_labels_dir = os.path.join(base_dir, 'labels', 'val')


**Load Images**

In [7]:
def load_images(image_dir, target_size=(224, 224)):
    images = []
    image_filenames = sorted(os.listdir(image_dir))

    for image_file in image_filenames:
        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, target_size)
            image = image / 255.0  # Normalize to [0, 1]
            images.append(image)

    return np.array(images)

# Load train and validation images
X_train = load_images(train_images_dir)
X_val = load_images(val_images_dir)

print(f"Loaded {X_train.shape[0]} training images.")
print(f"Loaded {X_val.shape[0]} validation images.")


Loaded 738 training images.
Loaded 185 validation images.


**Load Labels**

In [8]:
def load_labels(label_dir):
    labels = []
    label_filenames = sorted(os.listdir(label_dir))

    for label_file in label_filenames:
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, 'r') as file:
            boxes = []
            for line in file:
                # Assuming each line is in the format: class_id x_min y_min width height
                parts = line.strip().split()
                class_id = int(parts[0])
                bbox = list(map(float, parts[1:]))  # Convert bbox to float values
                boxes.append([class_id] + bbox)  # Combine class_id with bbox info
            labels.append(boxes)  # Add all boxes for the image to the labels list

    return labels  # Return a list of lists

# Now load the train and validation labels
y_train = load_labels(train_labels_dir)
y_val = load_labels(val_labels_dir)

print(f"Number of training labels: {len(y_train)}")
print(f"Example of a training label: {y_train[0]}")  # Check one example of loaded label


Number of training labels: 738
Example of a training label: [[2, 0.543893, 0.609375, 0.041985, 0.041667], [5, 0.332061, 0.346354, 0.129771, 0.182292], [5, 0.568702, 0.479167, 0.351145, 0.427083]]


**Padding Labels for Consistency**

In [9]:
def pad_labels(labels, max_objects=10):
    padded_labels = []
    for boxes in labels:
        if len(boxes) > max_objects:
            boxes = boxes[:max_objects]  # Trim to max_objects if more objects
        else:
            # Pad with zeros for both class and bbox if fewer objects
            boxes += [[0, 0, 0, 0, 0]] * (max_objects - len(boxes))
        padded_labels.append(boxes)
    return np.array(padded_labels)

# Pad the training and validation labels
y_train_padded = pad_labels(y_train)
y_val_padded = pad_labels(y_val)

print(f"Shape of padded training labels: {y_train_padded.shape}")


Shape of padded training labels: (738, 10, 5)


**Define the Model**

In [25]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model

input_layer = Input(shape=(224, 224, 3))

x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)

# Set explicit names for output layers
class_output = Dense(70, name='class_output')(x)
bbox_output = Dense(40, name='bbox_output')(x)

model = Model(inputs=input_layer, outputs=[class_output, bbox_output])
model.compile(optimizer='adam',
              loss={'class_output': 'categorical_crossentropy', 'bbox_output': 'mse'},
              metrics={'class_output': 'accuracy', 'bbox_output': 'mae'})


# Print model summary to confirm output names
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 222, 222, 32)   │            896 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_6           │ (None, 111, 111, 32)   │              0 │ conv2d_6[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 109, 109, 64)   │         18,496 │ max_pooling2d_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_7           │ (None, 54, 54, 64)     │              0 │ conv2d_7[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 52, 52, 128)    │         73,856 │ max_pooling2d_7[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_8           │ (None, 26, 26, 128)    │              0 │ conv2d_8[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 86528)          │              0 │ max_pooling2d_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 512)            │     44,302,848 │ flatten_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ class_output (Dense)      │ (None, 70)             │         35,910 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bbox_output (Dense)       │ (None, 40)             │         20,520 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 44,452,526 (169.57 MB)

 Trainable params: 44,452,526 (169.57 MB)

 Non-trainable params: 0 (0.00 B)

**Prepare the Data for Training**

In [34]:
train_class_labels = [box[0] for boxes in y_train_padded for box in boxes]
val_class_labels = [box[0] for boxes in y_val_padded for box in boxes]

# Get unique class labels
unique_train_labels = np.unique(train_class_labels)
unique_val_labels = np.unique(val_class_labels)

print("Unique class labels in training set:", unique_train_labels)
print("Unique class labels in validation set:", unique_val_labels)

# Update num_classes based on the number of unique classes in your dataset
num_classes = len(unique_train_labels)  # This will be the correct number of classes

# Split the padded labels into class and bounding box labels
y_train_classes = tf.keras.utils.to_categorical(y_train_padded[:, :, 0], num_classes=num_classes)
y_train_bboxes = y_train_padded[:, :, 1:]

y_val_classes = tf.keras.utils.to_categorical(y_val_padded[:, :, 0], num_classes=num_classes)
y_val_bboxes = y_val_padded[:, :, 1:]

# Print shapes before reshaping to verify
print(f"Before reshaping:")
print(f"Training class labels shape: {y_train_classes.shape}")
print(f"Training bounding box labels shape: {y_train_bboxes.shape}")
print(f"Validation class labels shape: {y_val_classes.shape}")
print(f"Validation bounding box labels shape: {y_val_bboxes.shape}")

# Flatten the class labels for training and validation
y_train_classes_flat = y_train_classes.reshape(-1, num_classes)  # (num_samples, num_classes)
y_val_classes_flat = y_val_classes.reshape(-1, num_classes)      # (num_samples, num_classes)

# Reshape bounding boxes if necessary
# Ensure bounding boxes match the number of class labels
num_samples_train = y_train_classes_flat.shape[0]
num_samples_val = y_val_classes_flat.shape[0]

y_train_bboxes_flat = y_train_bboxes.reshape(num_samples_train, -1)  # (num_samples, 10 * 4) or however many bbox coordinates
y_val_bboxes_flat = y_val_bboxes.reshape(num_samples_val, -1)        # (num_samples, 10 * 4)

# Print final shapes to verify
print(f"Final shapes after flattening:")
print(f"Training class labels shape: {y_train_classes_flat.shape}")
print(f"Training bounding box labels shape: {y_train_bboxes_flat.shape}")
print(f"Validation class labels shape: {y_val_classes_flat.shape}")
print(f"Validation bounding box labels shape: {y_val_bboxes_flat.shape}")



Unique class labels in training set: [0. 1. 2. 3. 4. 5. 6.]
Unique class labels in validation set: [0. 1. 2. 3. 4. 5. 6.]
Before reshaping:
Training class labels shape: (738, 10, 7)
Training bounding box labels shape: (738, 10, 4)
Validation class labels shape: (185, 10, 7)
Validation bounding box labels shape: (185, 10, 4)
Final shapes after flattening:
Training class labels shape: (7380, 7)
Training bounding box labels shape: (7380, 4)
Validation class labels shape: (1850, 7)
Validation bounding box labels shape: (1850, 4)


**Train the Model**

In [35]:
history = model.fit(
    X_train,
    {'class_output': y_train_classes_flat, 'bbox_output': y_train_bboxes_flat},
    validation_data=(X_val, {'class_output': y_val_classes_flat, 'bbox_output': y_val_bboxes_flat}),
    epochs=10,  # Increase epochs as needed
    batch_size=16
)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 738
'y' sizes: 7380, 7380
